In [56]:

#Imports 
import numpy as np #Represent ndarrays a.k.a. tensors
import matplotlib.pyplot as plt #For plotting
np.random.seed(0) #For repeatability of the experiment
import pickle #To read data for this experiment
import _pickle as cPickle
import gzip

# Third-party libraries
import numpy as np

#Setup
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

**Data**

In [57]:
#Read data
def load_data():
    with gzip.open('mnist.pkl.gz', 'rb') as f:
        training_data, validation_data, test_data = cPickle.load(f, encoding="latin-1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = np.array(list(zip(training_inputs, training_results)))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = np.array(list(zip(validation_inputs, va_d[1])))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = np.array(list(zip(test_inputs, te_d[1])))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

training_data, validation_data, test_data = load_data_wrapper()

training_data_nolab = np.delete(training_data,[1],axis = 1)
training_labels = np.array(np.delete(training_data,[0],axis=1))
training_labels_list = []
training_data_nolab_list = np.zeros(shape=(50000,784))

for i in range(len(training_labels)):
    list.append(training_labels_list, np.argmax(training_labels[i,0]))

for i in range(len(training_data_nolab)):
    training_data_nolab_list[i] = np.array(np.ravel(training_data_nolab[i,0]))

X = training_data_nolab_list
y = np.squeeze(training_labels_list)

test_data_nolab = np.delete(test_data,[1],axis = 1)
test_labels = np.array(np.delete(test_data,[0],axis=1))
test_labels_list = []
test_data_nolab_list = np.zeros(shape=(10000,784))

for i in range(len(test_labels)):
    list.append(test_labels_list, np.argmax(test_labels[i,0]))

for i in range(len(test_data_nolab)):
    test_data_nolab_list[i] = np.array(np.ravel(test_data_nolab[i,0]))

X_test = test_data_nolab_list
y_test = np.squeeze(test_labels_list)

valid_data_nolab = np.delete(validation_data,[1],axis = 1)
valid_labels = np.array(np.delete(validation_data,[0],axis=1))
valid_labels_list = []
valid_data_nolab_list = np.zeros(shape=(10000,784))

for i in range(len(valid_labels)):
    list.append(valid_labels_list, np.argmax(valid_labels[i,0]))

for i in range(len(valid_data_nolab)):
    valid_data_nolab_list[i] = np.array(np.ravel(valid_data_nolab[i,0]))

X_valid = valid_data_nolab_list
y_valid = np.squeeze(valid_labels_list)


**Model**

In [58]:
# Feedforward neural net model

# Start with an initial set of parameters randomly

D = X.shape[1] #Number of features
K = 10 #Number of classes assuming class index starts from 0

h = 100 # size of hidden layer
W = 0.01 * np.random.randn(D,h)
b = np.zeros((1,h))
W2 = 0.01 * np.random.randn(h,K)
b2 = np.zeros((1,K))

# Initial values from hyperparameter
reg = 4e-1 # regularization strength
print("regularization =",reg)
#For simplicity, we will not optimize this using grid search here.

#optimizing the reg parameter
reg_values = list([0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1])
print(reg_values)

regularization = 0.4
[0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1]


In [59]:
#Perform batch SGD using manual backprop

#For simplicity we will take the batch size to be the same as number of examples
num_examples = X.shape[0]
print(num_examples)

#Initial value for the Gradient Descent Parameter
step_size = 0.8e-0 #Also called learning rate
#For simplicity, we will not hand tune this algorithm parameter as well.

print("reg param:",reg)
print("step size:",step_size)
#print("iterations:",gd_iters)

# gradient descent loop
for i in range(100):

    # evaluate class scores, [N x K]
    hidden_layer = np.maximum(np.dot(X, W) + b, 0.01*(np.dot(X, W) + b))
    scores = np.dot(hidden_layer, W2) + b2

    # compute the class probabilities
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]
    
    # compute the loss: average cross-entropy loss and regularization
    corect_logprobs = -np.log(probs[range(num_examples),y])
    data_loss = np.sum(corect_logprobs)/num_examples
    reg_loss = 0.5*reg*np.sum(W*W) + 0.5*reg*np.sum(W2*W2)
    loss = data_loss + reg_loss
    if i % 2 == 0:
        print ("iteration:",i, " loss:", loss)

    # compute the gradient on scores
    dscores = probs
    dscores[range(num_examples),y] -= 1
    dscores /= num_examples
        
    # backpropate the gradient to the parameters
    # first backprop into parameters W2 and b2
    dW2 = np.dot(hidden_layer.T, dscores)
    db2 = np.sum(dscores, axis=0, keepdims=True)
    
    # next backprop into hidden layer
    dhidden = np.dot(dscores, W2.T)
    var = 0.01*np.dot(dscores, W2.T)
    
    dhidden[hidden_layer <= 0] = 0.01*dhidden[hidden_layer <= 0]  

    ## final backprop
    dW = np.dot(X.T, dhidden)
    db = np.sum(dhidden, axis = 0, keepdims = True)

    ## adding reg to gradient
    dW2 += reg*W2
    dW += reg*W

    ## stepsize
    W += -step_size * dW
    W2 += -step_size * dW2
    b2 += -step_size * db2
    b += -step_size * db


#def test_ffn_leakyrelu_classifier(X_data, y_data, W, b, W2, b2):
# Post-training: evaluate model accuracy
hidden_layer = np.maximum(0.01*np.dot(X_valid, W) + b, np.dot(X_valid, W) + b) 
scores = np.dot(hidden_layer, W2) + b2
predicted_class = np.argmax(scores, axis = 1) ### scores with max conf
test_accuracy = (np.mean(predicted_class == y_valid))
print("accuracy: ",test_accuracy)
#return test_accuracy

50000
reg param: 0.4
step size: 0.8
iteration: 0  loss: 3.87803261521
iteration: 2  loss: 2.63697432479
iteration: 4  loss: 2.37269566041
iteration: 6  loss: 2.31564989829
iteration: 8  loss: 2.3026767213
iteration: 10  loss: 2.29892786599
iteration: 12  loss: 2.29690994518
iteration: 14  loss: 2.29501279598
iteration: 16  loss: 2.29295119088
iteration: 18  loss: 2.2908127449
iteration: 20  loss: 2.28877976781
iteration: 22  loss: 2.28702693491
iteration: 24  loss: 2.2858718787
iteration: 26  loss: 2.28566593166
iteration: 28  loss: 2.28523800365
iteration: 30  loss: 2.28475162655
iteration: 32  loss: 2.28418500008
iteration: 34  loss: 2.28322035603
iteration: 36  loss: 2.28229514209
iteration: 38  loss: 2.28144705506
iteration: 40  loss: 2.28071112497
iteration: 42  loss: 2.28000317029
iteration: 44  loss: 2.27936090466
iteration: 46  loss: 2.27874962366
iteration: 48  loss: 2.2782095759
iteration: 50  loss: 2.27771622123
iteration: 52  loss: 2.277261604
iteration: 54  loss: 2.2768566

**Post Training**

In [60]:
#def test_ffn_leakyrelu_classifier(X_data, y_data, W, b, W2, b2):
# Post-training: evaluate model accuracy
hidden_layer = np.maximum(0.01*np.dot(X_test, W) + b, np.dot(X_test, W) + b) 
scores = np.dot(hidden_layer, W2) + b2
predicted_class = np.argmax(scores, axis = 1) ### scores with max conf
test_accuracy = (np.mean(predicted_class == y_test))
print("accuracy: ",test_accuracy)
#return test_accuracy

accuracy:  0.4234
